In [1]:
import pandas as pd
import folium
import os

In [2]:
import pandas as pd
import folium
import os
import base64

# Load data
df = pd.read_csv("zfinal.csv")

# Center map
map_center = [df["Latitude"].mean(), df["Longitude"].mean()]
# m = folium.Map(location=map_center, zoom_start=6, tiles=None)

m = folium.Map(
    location=map_center,
    zoom_start=6,
    tiles="CartoDB positron"
)


# Add Stamen Terrain tile (parchment-map-style with shading)
# folium.TileLayer(
#     tiles="https://stamen-tiles.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png",
#     attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
#          'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. '
#          'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under ODbL.',
#     name="Stamen Terrain",
#     control=False
# ).add_to(m)

# Directory of god icons
icon_folder = "icons"

for _, row in df.iterrows():
    god = row["God"].strip().capitalize()
    place = row["Place"]

    icon_path_png = os.path.join(icon_folder, f"{god}.png")
    icon_path_jpg = os.path.join(icon_folder, f"{god}.jpeg")

    if os.path.exists(icon_path_png):
        icon_path = icon_path_png
    elif os.path.exists(icon_path_jpg):
        icon_path = icon_path_jpg
    else:
        print(f"⚠️ Missing icon for: {god}")
        continue

    # Embed image using base64
    with open(icon_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
        ext = os.path.splitext(icon_path)[1][1:]  # remove dot
        data_uri = f"data:image/{ext};base64,{encoded}"

    # Combined label and icon
    html = f"""
    <div style="text-align: center;">
        <div style="font-size: 12px; font-weight: bold; color: black;">{place}</div>
        <img src="{data_uri}" style="width: 30px; height: 30px;">
    </div>
    """

    popup_text = (
        f"<b>{place}</b><br>"
        f"God: {god}<br>"
        f"Source: {row['Source']}<br>"
        f"Frequency: {row['Frequency']}"
    )

    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        icon=folium.DivIcon(html=html),
        popup=popup_text
    ).add_to(m)

# Save map
m.save("myth_map.html")
print("✅ Antique-style map with god icons and labels saved as 'myth_map.html'")


✅ Antique-style map with god icons and labels saved as 'myth_map.html'
